In [1]:
import pickle
import re

from tqdm.auto import tqdm

## Merge files and remove invalid data

In [2]:
film_dict = {}
p = re.compile(r"(?:subject/)(\d+)")

In [18]:
# change filename to add all information into a single dictionary
with open("animated_film_info.pickle", "rb") as file:
    raw = pickle.load(file)
    for film in raw:
        url = film.get('page_link')
        fid = int(p.findall(url)[0])
        
        if film.get('全名') is None:
            continue
        
        if film.get('剧情简介') is None:
            print("{} does not have a summary, removing...".format(film['全名']))
            continue

        if film.get('年份') is None:
            print("{} does not have year information, removing...".format(film['全名']))
            continue

        if len(film.get('影评')) != 5:
            print("{} does not have enough reviews, removing...".format(film['全名']))
            continue

        film_dict[fid] = film

蜡笔小新之黑暗珠珠大追击 クレヨンしんちゃん 暗黒タマタマ大追跡 does not have a summary, removing...
我的妹妹小桃子 もも子、かえるの歌がきこえるよ。 does not have a summary, removing...
欢乐树的朋友们 Happy Tree Friends does not have a summary, removing...
守望先锋：英雄 Overwatch: Hero does not have a summary, removing...
欢乐树的朋友们：赶尽杀绝 Happy Tree Friends: Overkill does not have a summary, removing...
鲁邦三世 东方见闻录 ～Another Page～ ルパン三世 東方見聞録 ～アナザーページ～ does not have a summary, removing...
新网球王子OVA1 邀请前夜 新テニスの王子様 OVA1 招聘前夜 does not have a summary, removing...
新网球王子OVA2 神之子vs皇帝 新テニスの王子様 OVA2 神の子vs皇帝 does not have a summary, removing...
空之境界 俯瞰风景3D 劇場版 空の境界 俯瞰風景3D does not have a summary, removing...
黄金拼图 Pretty Days きんいろモザイク Pretty Days does not have a summary, removing...
阿曼动画之超级无敌掌门狗系列1 Wallace & Gromit: The Aardman Collection does not have a summary, removing...
鲁邦三世 Master File ルパン三世 Master File does not have a summary, removing...
新网球王子OVA3 继承传说的人们 新テニスの王子様 OVA3 伝説を継ぐ者たち does not have a summary, removing...
结婚蛋糕 Wedding Cake does not have a summary, remo

In [17]:
for idx, film in enumerate(raw):
    if film.get('影评') is None:
        print(idx)

{'title': '龙猫',
 'page_link': 'https://movie.douban.com/subject/1291560/',
 'rating': '9.2',
 'cover': 'https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2540924496.jpg',
 '导演': '宫崎骏',
 '编剧': '宫崎骏',
 '主演': [{'name': '日高法子',
   'page_link': 'https://movie.douban.com/celebrity/1019382/'},
  {'name': '坂本千夏', 'page_link': 'https://movie.douban.com/celebrity/1025582/'},
  {'name': '糸井重里', 'page_link': 'https://movie.douban.com/celebrity/1379738/'},
  {'name': '岛本须美', 'page_link': 'https://movie.douban.com/celebrity/1037668/'},
  {'name': '北林谷荣', 'page_link': 'https://movie.douban.com/celebrity/1020128/'},
  {'name': '高木均', 'page_link': 'https://movie.douban.com/celebrity/1033543/'},
  {'name': '雨笠利幸', 'page_link': 'https://movie.douban.com/celebrity/1379740/'},
  {'name': '丸山裕子', 'page_link': 'https://movie.douban.com/celebrity/1031175/'},
  {'name': '广濑正志', 'page_link': 'https://movie.douban.com/celebrity/1043679/'},
  {'name': '鹫尾真知子',
   'page_link': 'https://movie.douban.com/c

In [21]:
# save the consolidated data
with open("film_dict.pickle", "wb") as file:
    pickle.dump(film_dict, file)

In [50]:
# open if necessary
with open("film_dict.pickle", "rb") as file:
    film_dict = pickle.load(file)

In [51]:
len(raw)

1999

In [52]:
len(film_dict)

1933

## Extract actor data

In [22]:
actor_names = {actor['name'] for info in film_dict.values() for actor in info['主演']}
name_to_id = {name: idx+1000000 for idx, name in enumerate(actor_names)}

In [23]:
# save the conversion data
with open("actor_name_to_id.pickle", "wb") as file:
    pickle.dump(name_to_id, file)

In [53]:
actors_dict = {}
p = re.compile(r"(?:celebrity/)(\d+)")
new_id = 1000000
for film_id, info in tqdm(film_dict.items()):
    for actor in info['主演']:
        actor_id = name_to_id[actor['name']]
        if actor_id not in actors_dict:
            actor['works'] = {film_id}
            actors_dict[actor_id] = actor
        else:
            actors_dict[actor_id]['works'].add(film_id)
#             if 'celebrity' in actor['page_link']:
#                 actors_dict[actor_id]['page_link'] = actor['page_link']

In [54]:
aurl = []
for actor_id, info_dict in actors_dict.items():
    aurl.append(info_dict.get('page_link'))

In [60]:
problematic_actors = {actor_id : info_dict for actor_id,info_dict in actors_dict.items() if 'search' in info_dict['page_link']}

In [65]:
problematic_actors

{1001231: {'name': '中村祐子',
  'page_link': 'https://movie.douban.com/subject_search?search_text=%E4%B8%AD%E6%9D%91%E7%A5%90%E5%AD%90',
  'works': {2043546}},
 1003464: {'name': '辰己努',
  'page_link': 'https://movie.douban.com/subject_search?search_text=%E8%BE%B0%E5%B7%B1%E5%8A%AA',
  'works': {1293318}},
 1002098: {'name': '白石绫乃',
  'page_link': 'https://movie.douban.com/subject_search?search_text=%E7%99%BD%E7%9F%B3%E7%BB%AB%E4%B9%83',
  'works': {1293318}},
 1004390: {'name': '志乃原良子',
  'page_link': 'https://movie.douban.com/subject_search?search_text=%E5%BF%97%E4%B9%83%E5%8E%9F%E8%89%AF%E5%AD%90',
  'works': {1293318}},
 1001747: {'name': '山口朱美',
  'page_link': 'https://movie.douban.com/subject_search?search_text=%E5%B1%B1%E5%8F%A3%E6%9C%B1%E7%BE%8E',
  'works': {1293318}},
 1002543: {'name': '端田宏三',
  'page_link': 'https://movie.douban.com/subject_search?search_text=%E7%AB%AF%E7%94%B0%E5%AE%8F%E4%B8%89',
  'works': {1293318}},
 1001626: {'name': '桂歌若',
  'page_link': 'https://movie.do

In [63]:
# save the consolidated data
with open("problem_actor_dict.pickle", "wb") as file:
    pickle.dump(problematic_actors, file)

In [27]:
for actor_id, info in tqdm(actors_dict.items()):
    if info.get('collaboration') is None:
        info['collaboration'] = {}
    
    for other_id, other_info in actors_dict.items():
        if actor_id == other_id:
            continue
        if info['collaboration'].get(other_id) is None:
            colab_works = info['works'].intersection(other_info['works'])
            if colab_works:
                info['collaboration'][other_id] = colab_works
                if other_info.get('collaboration') is None:
                    other_info['collaboration'] = {}
                other_info['collaboration'][actor_id] = colab_works

In [37]:
list(actors_dict.items())[:-1]

[(1004747,
  {'name': '金妮弗·古德温',
   'page_link': 'https://movie.douban.com/celebrity/1017930/',
   'works': {25662329, 26258872},
   'collaboration': {1005215: {25662329},
    1001417: {25662329},
    1003524: {25662329},
    1004868: {25662329},
    1004083: {25662329},
    1004038: {25662329},
    1003703: {25662329},
    1004583: {25662329},
    1001122: {25662329},
    1000083: {26258872},
    1000622: {26258872},
    1001804: {26258872},
    1003417: {26258872},
    1003489: {26258872},
    1002079: {26258872},
    1004438: {26258872},
    1000468: {26258872},
    1000441: {26258872}},
   '性别': '女',
   '星座': '双子座',
   '出生日期': '1978-05-22',
   '出生地': '美国,田纳西州,孟菲斯',
   '职业': '演员 / 配音 / 导演',
   '更多外文名': 'Jennifer Michelle Goodwin (本名) / Gigi (昵称) / Ginny (昵称)',
   '更多中文名': '詹妮弗·古德温',
   '家庭成员': 'Josh Dallas(丈夫）',
   'imdb编号': 'nm0329481',
   '全名': '金妮弗·古德温 Ginnifer Goodwin',
   '影人简介': '金妮弗·古德温，美国演员。1996年毕业于洛桑大学学院，此后在汉诺威大学主修戏剧。一年后，转学到波士顿大学美术学院，在2001年，她以优异成绩获得了学士学位。 2001年，Goodwin出演的第一

In [13]:
max_len = 0
keyname = '更多中文名'
for k,v in actors_dict.items():
    if v.get(keyname) is not None:
        curr_len = len(v[keyname])
        if curr_len > max_len:
            max_len = curr_len
            
print(max_len)

96


In [61]:
# save the consolidated data
with open("actor_dict.pickle", "wb") as file:
    pickle.dump(actors_dict, file)

In [12]:
# open the consolidated data
with open("actor_dict.pickle", "rb") as file:
    actors_dict = pickle.load(file)

## Extract comment data

In [39]:
comment_dict = {}
for film_name, info in tqdm(film_dict.items()):
    for comment in info['影评']:
        cid = int(comment['id'])
        comment['film_id'] = film_name
        comment_dict[cid] = comment

In [40]:
# save the consolidated data
with open("comments_database.pickle", "wb") as file:
    pickle.dump(comment_dict, file)

In [15]:
# open the consolidated data
with open("comments_database.pickle", "rb") as file:
    comment_dict = pickle.load(file)

In [20]:
import random
random.choice(list(comment_dict.items()))

(1092174597,
 {'id': '1092174597',
  'user': '吉米小島',
  'time': '2016-10-02',
  'text': '从想改变自己的心情开始，就站在起跑线上了。喜欢一个人多么美好，热乎乎的狗粮吃饱了也往肚子里塞。',
  'film_id': 26546802})

In [22]:
comment_dict[488175234]

{'id': '488175234',
 'user': '寂地',
 'time': '2012-01-28',
 'text': '相当和谐的主题和世界观，经济与环境的矛盾、保护与破坏的冲突，人类将会走向什么未来？世界末日来临之前，大自然的绿色仍旧在为地球付出，为人类服务。当娜乌西卡公主在黄金色的草原上时，我感动死了...久石让的配乐部部很赞。',
 'film_id': 1291585}

In [10]:
max_len = 0
keyname = 'text'
for k,v in comment_dict.items():
    if v.get(keyname) is not None:
        curr_len = len(v[keyname])
        if curr_len > max_len:
            max_len = curr_len
            
print(max_len)

203


## Removing extracted data from DB

In [32]:
for film_name, info in tqdm(film_dict.items()):
    info['影评'] = [int(comment['id']) for comment in info['影评']]
    info['主演'] = [name_to_id[actor['name']] for actor in info['主演']]

In [17]:
import random
random.choice(list(film_dict.items()))

(25825755,
 {'title': '羽国之旅',
  'rating': '7.6',
  'cover': 'https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2178229277.jpg',
  '导演': '埃斯本·托夫特·雅克伯森',
  '编剧': '埃斯本·托夫特·雅克伯森 / 雅尼克·泰·莫索特',
  '主演': [1004202,
   1002703,
   1001833,
   1003403,
   1005284,
   1003039,
   1002993,
   1001117,
   1000546],
  '类型': '动画 / 冒险',
  '官方网站': 'beyondbeyond.copenhagenbombay.com',
  '制片国家/地区': '瑞典 / 丹麦',
  '语言': '瑞典语',
  '上映日期': '2014-02-10(柏林电影节) / 2014-03-21(瑞典)',
  '片长': '78分钟',
  '又名': '小兔历险记 / 怪物王国 / Beyond Beyond',
  'IMDb链接': 'tt3480110',
  '全名': '羽国之旅 Resan till Fjäderkungens Rike',
  '年份': '(2014)',
  '剧情简介': '这是一个关于等待不可能的故事。这是一只年幼的小兔子，他还不懂得生命的规则，试图与最强大的势力斗争。随着他的长大，他对生命了解得越来越多。',
  '影评': [999684850, 994765897, 994679254, 996173862, 1201035437]})

In [54]:
# save the consolidated data
with open("film_database_final.pickle", "wb") as file:
    pickle.dump(film_dict, file)

In [4]:
# save the consolidated data
with open("film_database_final.pickle", "rb") as file:
    film_dict = pickle.load(file)

In [34]:
max_len = 0
keyname = '剧情简介'
for k,v in film_dict.items():
    if v.get(keyname) is not None:
        curr_len = len(v[keyname])
        if curr_len > max_len:
            max_len = curr_len
            
print(max_len)

383
